In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/genius-song-lyrics-with-language-information/song_lyrics.csv


https://www.kaggle.com/datasets/carlosgdcj/genius-song-lyrics-with-language-information

This dataset originates from the website 'Genius' where lyrics to songs are posted as well as community-submitted explanations to the meanings behind those lyrics. The dataset has song titles, the full string of lyrics, and a few other columns. However, the column of note this dataset has is a `tag` field which stores which genre each song belongs to. (i.e. rap, pop, rock, etc.)

For my project, I would like to train an NLP AI to try to find a connection between lyrics and genre. I'd like to train it to classify the lyrics of a song into which genre they belong to. There is not necessarily any innate connection between these two concepts, which is why I especially want to see how the AI tackles this problem. Perhaps it will be able to find some connection that I am unable to see.

In [2]:
lyric_database = pd.read_csv("/kaggle/input/genius-song-lyrics-with-language-information/song_lyrics.csv")
lyric_database.head()

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",1,en,en,en
1,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",3,en,en,en
2,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4,en,en,en
3,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,5,en,en,en
4,Fly In,rap,Lil Wayne,2005,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",6,en,en,en


This dataset has a few problems that need to be addressed before using it for this project. Firstly, this dataset has lyrics from all different languages. For the purpose of this project, I'm going to limit the scope to just English songs. There may be a deeper correlation that can be made by using all languages, but for readability I chose to limit it to English. This can be easily done by filtering out songs using the `language` column restricting it to only 'en'.

Some of the data entries lack a tag or lack substantial lyrics which would break or confuse the AI so those will also be removed from the database. 

Additionally, the lyrics of a song on Genius are split up by verses and choruses and other musical notations separated from the lyrics with brackets. For example, a song can have the line `[Verse 1: Drake]`that follows before a verse sung by Drake. For the purposes of this project, those lines will remain in the lyrics. They aren't technically lyrics, but the AI can notice patterns with artists and verse/chorus/introduction setups to help classify which genre the song is in.

Finally, Genius allows users to upload lyrics for things that aren't songs as well such as scripts or poems. These are all tagged as 'misc,' but some songs are also tagged as misc. For the sake of simplicity, I will be culling all lyrics tagged as misc from the database even if they are songs.

In [3]:
db = (
    lyric_database
    .loc[lyric_database["language"] == "en"]
    .loc[lyric_database["tag"] != "misc"]
    .loc[lyric_database["tag"] != ""]
    .loc[lyric_database["tag"] != None]
    .loc[lyric_database["lyrics"] != None]
    .loc[lyric_database["lyrics"] != ""]
    .loc[lyric_database["lyrics"] != "[Instrumental]"] #some songs without lyrics will squeeze
                                                       #their way onto the website for no
                                                       # discernable reason
)

This dataset has several columns that can be used for all sorts of implementation. For the sake of this project only the `lyrics`, `tag`, and `id` columns are of interest to the AI, and `title` and `artist` help *us* read and understand the data.

`id` is how we iterate through each song, and `title` and `artist` is how we display the name of the song found by the id.

`lyrics` is our text input and `tag` is our classification output. We need these to train the AI.

The `language` column is extraneous now that we have filtered out any songs that are not in english.

In [4]:
columns_of_interest = ['id', 'title','artist', 'lyrics', 'tag']
db = db[columns_of_interest]
db.head()

,id,title,artist,lyrics,tag
0,1,Killa Cam,Cam'ron,"[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",rap
1,3,Can I Live,JAY-Z,"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",rap
2,4,Forgive Me Father,Fabolous,Maybe cause I'm eatin\nAnd these bastards fien...,rap
3,5,Down and Out,Cam'ron,[Produced by Kanye West and Brian Miller]\n\n[...,rap
4,6,Fly In,Lil Wayne,"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",rap


Once we've culled all unnecessary data and extraneous columns from the database we are left with a cleaned up set of data to feed to the AI model. However, there are still foreign symbols and other quirks present in the data that will confuse the AI. 

Most notably, each of the string values has many many `\n`s signifying new lines in the lyrics page. We want to keep these new-line symbols for the AI to learn from, but they are not separated by spaces and the AI will consider anything not separated by spaces as one word. Consider this song for example:

In [5]:
print(db.loc[db["id"] == 154314].values)

[[154314 'Do I Wanna Know?' 'Arctic Monkeys'
  "[Verse 1]\nHave you got colour in your cheeks?\nDo you ever get that fear that you can't shift the type\nThat sticks around like summat in your teeth?\nAre there some aces up your sleeve?\nHave you no idea that you're in deep?\nI've dreamt about you nearly every night this week\nHow many secrets can you keep?\n'Cause there's this tune I found\nThat makes me think of you somehow an' I play it on repeat\nUntil I fall asleep, spillin' drinks on my settee\n\n[Pre-Chorus]\n(Do I wanna know?) If this feelin' flows both ways?\n(Sad to see you go) Was sorta hopin' that you'd stay\n(Baby, we both know) That the nights were mainly made\nFor sayin' things that you can't say tomorrow day\n\n[Chorus]\nCrawlin' back to you\nEver thought of callin' when\nYou've had a few?\n'Cause I always do\nMaybe I'm too\nBusy bein' yours\nTo fall for somebody new\nNow, I've thought it through\nCrawlin' back to you\n[Verse 2]\nSo have you got the guts?\nBeen wonderin'

In this example you can see there are many pieces of the string that are *supposed* to be separate lines, but because of the special character \n being present and not being buffered by spaces words like "`settee\n\n[Pre-Chorus]\n(Do`" are made. This will completely butcher a one-hot matrix and confuse our AI. To fix this, we can separate `\n` by whitespaces to separate it from the rest of the words. 

By just doing this we would separate our word example into "`settee \n \n [Pre-Chorus] \n (Do`." This still leaves words like "`(Do`" and "`[Verse`" present in our example. To solve this, boundary symbols like parentheses and brackets need to be separated by spaces too. By implementing this fix we can keep the musical notation and line separation for the AI to learn from without breaking the AI completely.

To do this adjustement we will be using a regular expression library `re`. Here is what our lyrics will look like after we split these character up with whitespace:

In [6]:
import re
lyrics = db.loc[db["id"] == 154314].values[0][3]

parts = re.split(r"([\n\[\]\(\)])", lyrics)
#print(repr(parts))

whitespace_lyrics = " ".join(filter(None, parts))
print(repr(whitespace_lyrics))

"[ Verse 1 ] \n Have you got colour in your cheeks? \n Do you ever get that fear that you can't shift the type \n That sticks around like summat in your teeth? \n Are there some aces up your sleeve? \n Have you no idea that you're in deep? \n I've dreamt about you nearly every night this week \n How many secrets can you keep? \n 'Cause there's this tune I found \n That makes me think of you somehow an' I play it on repeat \n Until I fall asleep, spillin' drinks on my settee \n \n [ Pre-Chorus ] \n ( Do I wanna know? )  If this feelin' flows both ways? \n ( Sad to see you go )  Was sorta hopin' that you'd stay \n ( Baby, we both know )  That the nights were mainly made \n For sayin' things that you can't say tomorrow day \n \n [ Chorus ] \n Crawlin' back to you \n Ever thought of callin' when \n You've had a few? \n 'Cause I always do \n Maybe I'm too \n Busy bein' yours \n To fall for somebody new \n Now, I've thought it through \n Crawlin' back to you \n [ Verse 2 ] \n So have you got

Much better! Now you can see our original problem "word" of "`settee\n\n[Pre-Chorus]\n(Do`" has been properly split into "`sette \n \n [ Pre-Chorus ] ( Do`". Our data still is not perfect, as you can see there are puncuation symbols like `?` and `,`  and any punctuated word will be considered a separate entry by the AI. So `Know` and `Know?` and `Know,` are all considered unique words. This can be solved by removing the punctuation from our lyrics using `re`. 

In [7]:
unpunctuated = re.sub(r"([?\.\,\!])",'',whitespace_lyrics)
print(repr(unpunctuated))

"[ Verse 1 ] \n Have you got colour in your cheeks \n Do you ever get that fear that you can't shift the type \n That sticks around like summat in your teeth \n Are there some aces up your sleeve \n Have you no idea that you're in deep \n I've dreamt about you nearly every night this week \n How many secrets can you keep \n 'Cause there's this tune I found \n That makes me think of you somehow an' I play it on repeat \n Until I fall asleep spillin' drinks on my settee \n \n [ Pre-Chorus ] \n ( Do I wanna know )  If this feelin' flows both ways \n ( Sad to see you go )  Was sorta hopin' that you'd stay \n ( Baby we both know )  That the nights were mainly made \n For sayin' things that you can't say tomorrow day \n \n [ Chorus ] \n Crawlin' back to you \n Ever thought of callin' when \n You've had a few \n 'Cause I always do \n Maybe I'm too \n Busy bein' yours \n To fall for somebody new \n Now I've thought it through \n Crawlin' back to you \n [ Verse 2 ] \n So have you got the guts \

Finally, to get our data the most AI friendly we should fix slang, regulate capital letters, and lemmatize all words in each song. This gives our AI the highest chance of success and also allows us to more accurately analyze our dataset.

Some slang we can't fix or predict, but words that end in an apostrophe can be replaced with a 'g', like `Crawlin'` to `Crawling` in our example above. Additionally, `'Cause` can be changed to `Because`. That should affect a significant amount of songs, but it will be impossible to account for all slang. This is good enough for our purposes.

To remove capital letters we can just use python's built in `.lower()` method. Very simple.

To lemmatize the words in our dataset I will be using 

In [8]:
#code to find a song by its title and artist
for item in (db.loc[db["title"] == ""].values):
    for x in item:
        print(f"{x}\n")

In [9]:
db.sort_values(by="id")
#the IDs go all the way up to 7.8 million despite only having 5.1 million songs in the database

,id,title,artist,lyrics,tag
0,1,Killa Cam,Cam'ron,"[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",rap
1,3,Can I Live,JAY-Z,"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",rap
2,4,Forgive Me Father,Fabolous,Maybe cause I'm eatin\nAnd these bastards fien...,rap
3,5,Down and Out,Cam'ron,[Produced by Kanye West and Brian Miller]\n\n[...,rap
4,6,Fly In,Lil Wayne,"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",rap
...,...,...,...,...,...
5134847,7882838,Everything Is Alright Now,Chuck Bernard,"Everything is alright now\nOh yes, baby\nEvery...",pop
5134849,7882840,White Lies,ElementD,[Verse 1]\nHalf truth and half you\nDidn't we ...,pop
5134851,7882842,Ocean,Effemar,[Verse 1]\nDance for me now\nKeeping yourself ...,pop
5134853,7882845,Raise Our Hands,"Culture Code, Pag & Mylo",[Verse 1]\nHere our purpose feels alive\nWe ar...,pop
